In [109]:
import sys
import os
os.chdir('../')


import pandas as pd
import numpy as np
 
from tqdm import tqdm


from file_cache.utils.util_pandas import *
from file_cache.cache import file_cache
from functools import lru_cache
from glob import glob

%matplotlib inline
from core.conf import *
from spider.mi import *
!pwd

2019-09-06 13:34:14,458 pyplot.py[225] DEBUG Loaded backend module://ipykernel.pylab.backend_inline version unknown.


/Users/lali2/Documents/workspace_py


In [167]:


def get_data_from_gg(name, lang='zh_CN'): #'en_US']
    try:
        from requests.utils import quote
        name_new = quote(name)
        url = f'https://play.google.com/store/search?q={name_new}&c=apps&hl=zh_CN'
        from lxml import html
        import requests
        print(url)
        url2=None
        response = requests.get(url)
        #print(response.content)
        tree = html.fromstring(response.content)
        link = tree.xpath('//div[@class="Vpfmgd"]//div[@class="wXUyZd"]//a')[0]
        #for lang in ['zh_CN', 'en_US']:
        url2 = f'https://play.google.com{link.get("href")}&hl={lang}'
        print('url2==',url2)
        response = requests.get(url2)
        tree = html.fromstring(response.text)

        # #desc = tree.xpath('//*[@id="doc"]//p[@class="content content_hover"]/text()')
        desc = tree.xpath('//div[@itemprop="description"]//*/text()')
        desc = ','.join(desc)


        desc = desc.replace('\t', '')
        desc = desc.replace('\r', '')
        desc = desc.replace('\n', '').strip()
        print(desc)
        # dp = tree.xpath('//*[@id="doc"]//div/span[@class="head-content"]/text()')
        # dp = dp[0] if len(dp) > 0 else ''
        # dp = dp.strip()
        # print('dp===',dp)

        # #closed_ids = tree.xpath('/html/body/div[2]/div[2]/div[2]/div[2]/div[3]/ol/li[*]/a[1]')  # [0]

        match_name = tree.xpath('//h1[@itemprop="name"]/span/text()')[0]
        print('match_name', match_name,  'name', name)
        cat_list = tree.xpath('//div[@class="qQKdcc"]/span/a/text()')
        cat_list = ','.join(cat_list)
        print('cat_list', cat_list)

        res = {
            #'wd_id': wd_id,
            'name': name,
            'match_name': str(match_name),
            #'tag_list': ','.join(tag_list),
            'cat_list': ','.join(cat_list),
            #'closed_ids': ','.join(closed_ids),
            #'path':     str(path),
            'desc':     desc,
            #'dp': dp.strip().replace('\t', ''),  # 点评
            'ct':pd.to_datetime('now'),
            'source':'gg'

        }

    except Exception as e:
        print(name)
        print(name, url2 or url)
        print(e)
        res = {'name': name,'ct':pd.to_datetime('now'), 'source':'gg'}

    #print(res)
    return res


name = '品炫娱乐城'

get_data_from_gg(name, lang='en_US')

2019-09-10 10:33:42,846 connectionpool.py[815] DEBUG Starting new HTTPS connection (1): play.google.com:443


https://play.google.com/store/search?q=%E5%93%81%E7%82%AB%E5%A8%B1%E4%B9%90%E5%9F%8E&c=apps&hl=zh_CN


2019-09-10 10:33:43,067 connectionpool.py[396] DEBUG https://play.google.com:443 "GET /store/search?q=%E5%93%81%E7%82%AB%E5%A8%B1%E4%B9%90%E5%9F%8E&c=apps&hl=zh_CN HTTP/1.1" 200 None
2019-09-10 10:33:43,353 connectionpool.py[815] DEBUG Starting new HTTPS connection (1): play.google.com:443


url2== https://play.google.com/store/apps/details?id=com.memoriki.fullhousecasino&hl=en_US


2019-09-10 10:33:43,603 connectionpool.py[396] DEBUG https://play.google.com:443 "GET /store/apps/details?id=com.memoriki.fullhousecasino&hl=en_US HTTP/1.1" 200 None


No.1 Multiplayer Casino Game in Asia,The NEW free to play Full House Casino has the most unique style of casino slots and classic table games! Get 1,500,000 welcome bonus, enjoy 70+ well-picked slots from Vegas and Macau Casinos, compete with real-time players and win huge JACKPOT! Experience top table games like Bingo, Blackjack, Texas Holdem and many more! Compete with your friends to become the legendary big fish winner in our Daily Tournament! With 5,000,000+ download worldwide & 500,000 5-star-rating, Full House Casino is definitely one of the best casino games in the world! Let's Spin Now!,CASINO GAMES PARTY FROM VEGAS & MACAU,Full House Casino guarantees over 60+ exotic and colorful casino slots originated from the most famous casinos in Vegas and Macau. Getting bored with the repetitive slots out there? Check out our original slots with stunning effects introduced every month! We share our best casino games with various themes such as mafia, steampunk, cartoon and many more!,PL

{'name': '品炫娱乐城',
 'match_name': 'Full House Casino - Free Vegas Slots Casino Games',
 'cat_list': 'M,E,2,O,N,,,C,a,s,i,n,o',
 'desc': 'No.1 Multiplayer Casino Game in Asia,The NEW free to play Full House Casino has the most unique style of casino slots and classic table games! Get 1,500,000 welcome bonus, enjoy 70+ well-picked slots from Vegas and Macau Casinos, compete with real-time players and win huge JACKPOT! Experience top table games like Bingo, Blackjack, Texas Holdem and many more! Compete with your friends to become the legendary big fish winner in our Daily Tournament! With 5,000,000+ download worldwide & 500,000 5-star-rating, Full House Casino is definitely one of the best casino games in the world! Let\'s Spin Now!,CASINO GAMES PARTY FROM VEGAS & MACAU,Full House Casino guarantees over 60+ exotic and colorful casino slots originated from the most famous casinos in Vegas and Macau. Getting bored with the repetitive slots out there? Check out our original slots with stunni

In [137]:


def get_data_from_bd(name):
    try:
        from requests.utils import quote
        name_new = quote(name)
        url2 = f'http://www.baidu.com/s?wd={name_new}%20安卓版%20安卓'
        print(url2)
        from lxml import html
        import requests
        #print(url2)
        response = requests.get(url2, timeout=5000)
        #print(1)
        # print(response.text)
        tree = html.fromstring(response.content)
        # print(tree)
        # print(url2)
        # print(response.text)
        # desc = tree.xpath('/html/body/div[2]/div[2]/div[2]/div[1]/div[3]/div/div/text()[1]'
        # desc = tree.xpath('/html/body/div[2]/div[2]/div[2]/div[1]/div/div/div[1]/p/text()[1]')
        desc = tree.xpath('//*[@id="1" or @id="2"]//div[@class="c-abstract"]//text()')
        #print(2)
        desc = ''.join(desc)
        import re
        desc = re.sub('[0-9]*年[0-9]*月[0-9]*日.-.', '', desc)
        #print(3)
        res = {
            'name': name,
            #'match_name': name,
            'desc': desc,
            'ct': pd.to_datetime('now'),

        }
    except Exception as e:
        print(name, url2)
        print(e)
        res = {'name': name, 'ct': pd.to_datetime('now'), }
    print(4)
    return res


get_data_from_bd('花娘宝盒')

2019-09-06 16:56:54,332 connectionpool.py[206] DEBUG Starting new HTTP connection (1): www.baidu.com:80


http://www.baidu.com/s?wd=%E8%8A%B1%E5%A8%98%E5%AE%9D%E7%9B%92%20安卓版%20安卓


2019-09-06 16:57:54,220 connectionpool.py[396] DEBUG http://www.baidu.com:80 "GET /s?wd=%E8%8A%B1%E5%A8%98%E5%AE%9D%E7%9B%92%20%E5%AE%89%E5%8D%93%E7%89%88%20%E5%AE%89%E5%8D%93 HTTP/1.1" 200 None


4


{'name': '花娘宝盒',
 'desc': '花娘宝盒app是最新推出的一款手机美女直播宝盒软件,这里有超多超火爆的直播平台,大家可以在线免费畅享,你所期待的直播表演全部都有,更有海量的美女主播与你实时互动...花娘宝盒直播是一款非常不错的看美女直播软件,这里全天候都有高颜值主播为你带来精彩的才艺表演,更有礼物、连麦等互动玩法让你和主播的距离更近,花娘宝盒直播让...',
 'ct': Timestamp('2019-09-06 08:57:54.661814')}

In [108]:

def get_data_from_bdsj(name):
    try:
        from requests.utils import quote
        name_new = quote(name)
        url = f'https://shouji.baidu.com/s?wd={name_new}&data_type=app&f=header_all%40input%40btn_search'
        from lxml import html
        import requests
        print(url)
        url2=None
        response = requests.get(url)
        #print(response.content)
        tree = html.fromstring(response.content)
        link = tree.xpath('//a[@class="app-name"]')[0]
        print('===',link)
        url2 = f'https://shouji.baidu.com{link.get("href")}'
        print('url2==',url2)
        response = requests.get(url2)
        tree = html.fromstring(response.content)

        desc = tree.xpath('//*[@id="doc"]//p[@class="content content_hover"]/text()')
        desc = ''.join(desc) if len(desc) > 0 else ''
        desc = desc.replace('\t', '')
        desc = desc.replace('\r', '')
        desc = desc.replace('\n', '').strip()
        print(desc)
        dp = tree.xpath('//*[@id="doc"]//div/span[@class="head-content"]/text()')
        dp = dp[0] if len(dp) > 0 else ''
        dp = dp.strip()
        print('dp===',dp)

        #closed_ids = tree.xpath('/html/body/div[2]/div[2]/div[2]/div[2]/div[3]/ol/li[*]/a[1]')  # [0]

        match_name = tree.xpath('//*[@id="doc"]/div[2]/div/div[1]/div/div[2]/h1/span/text()')[0]
        print('match_name', match_name)
        cat_list = tree.xpath('//*[@id="doc"]/div[1]/div//span/a/text()')[1:]
        print('cat_list', cat_list)

        res = {
            #'wd_id': wd_id,
            'name': name,
            'match_name': str(match_name),
            #'tag_list': ','.join(tag_list),
            'cat_list': ','.join(cat_list),
            #'closed_ids': ','.join(closed_ids),
            #'path':     str(path),
            'desc':     desc,
            'dp': dp.strip().replace('\t', ''),  # 点评
            'ct':pd.to_datetime('now'),
            'source':'360'

        }




    except Exception as e:
        print(name)
        print(name, url2 or url)
        print(e)
        res = {'name': name,'ct':pd.to_datetime('now'), }

    print(res)
    return res

desc = get_data_from_bdsj('游戏王座')
desc = get_data_from_bdsj('iLightpro')


2019-09-05 13:02:40,990 connectionpool.py[815] DEBUG Starting new HTTPS connection (1): shouji.baidu.com:443


https://shouji.baidu.com/s?wd=%E6%B8%B8%E6%88%8F%E7%8E%8B%E5%BA%A7&data_type=app&f=header_all%40input%40btn_search


2019-09-05 13:02:41,458 connectionpool.py[396] DEBUG https://shouji.baidu.com:443 "GET /s?wd=%E6%B8%B8%E6%88%8F%E7%8E%8B%E5%BA%A7&data_type=app&f=header_all%40input%40btn_search HTTP/1.1" 200 None
2019-09-05 13:02:41,515 connectionpool.py[815] DEBUG Starting new HTTPS connection (1): shouji.baidu.com:443


=== <Element a at 0x1282b4098>
url2== https://shouji.baidu.com/game/25980583.html


2019-09-05 13:02:42,235 connectionpool.py[396] DEBUG https://shouji.baidu.com:443 "GET /game/25980583.html HTTP/1.1" 200 None
2019-09-05 13:02:42,371 connectionpool.py[815] DEBUG Starting new HTTPS connection (1): shouji.baidu.com:443


由游戏王死忠玩家自费研发的手机网游版游戏王！~《游戏王座》使用说明书【成分】：由怪兽卡、魔法卡、陷阱卡合计数千张卡牌以及刺激性、耐久性玩法组成。【性状】：本游戏为真正集换式卡牌游戏(TCG)，随着玩家的深入而不断释放独特魅力。【功能主治】：锻炼思维，形成头脑风暴。对智商105以上者的智商有明显促进作用，对无脑游戏厌恶症、收集强迫症患者有治愈之疗效。【不良反应...由游戏王死忠玩家自费研发的手机网游版游戏王！~《游戏王座》使用说明书【成分】：由怪兽卡、魔法卡、陷阱卡合计数千张卡牌以及刺激性、耐久性玩法组成。【性状】：本游戏为真正集换式卡牌游戏(TCG)，随着玩家的深入而不断释放独特魅力。【功能主治】：锻炼思维，形成头脑风暴。对智商105以上者的智商有明显促进作用，对无脑游戏厌恶症、收集强迫症患者有治愈之疗效。【不良反应】：经临床实验，对智商90以下者的智商和自信有降低和灭杀反应。【注意事项】：使用本游戏时请务必仔细阅读游戏内的“新手入门”，同时请参考“不良反应”。试用后请自行判断自己是否适合继续使用。
dp=== 魔幻卡牌游戏
match_name 游戏王座
cat_list ['游戏', '角色扮演']
{'name': '游戏王座', 'match_name': '游戏王座', 'cat_list': '游戏,角色扮演', 'desc': '由游戏王死忠玩家自费研发的手机网游版游戏王！~《游戏王座》使用说明书【成分】：由怪兽卡、魔法卡、陷阱卡合计数千张卡牌以及刺激性、耐久性玩法组成。【性状】：本游戏为真正集换式卡牌游戏(TCG)，随着玩家的深入而不断释放独特魅力。【功能主治】：锻炼思维，形成头脑风暴。对智商105以上者的智商有明显促进作用，对无脑游戏厌恶症、收集强迫症患者有治愈之疗效。【不良反应...由游戏王死忠玩家自费研发的手机网游版游戏王！~《游戏王座》使用说明书【成分】：由怪兽卡、魔法卡、陷阱卡合计数千张卡牌以及刺激性、耐久性玩法组成。【性状】：本游戏为真正集换式卡牌游戏(TCG)，随着玩家的深入而不断释放独特魅力。【功能主治】：锻炼思维，形成头脑风暴。对智商105以上者的智商有明显促进作用，对无脑游戏厌恶症、收集强迫症患者有治愈之疗效。【不良反应】：经临床实验，对智商90以下者的智商和自信有降低和灭杀反应。【注意事项】：使用本游戏时请务必仔细阅读游戏内的“

2019-09-05 13:02:42,791 connectionpool.py[396] DEBUG https://shouji.baidu.com:443 "GET /s?wd=iLightpro&data_type=app&f=header_all%40input%40btn_search HTTP/1.1" 200 None
2019-09-05 13:02:42,847 connectionpool.py[815] DEBUG Starting new HTTPS connection (1): shouji.baidu.com:443


=== <Element a at 0x13244dae8>
url2== https://shouji.baidu.com/software/26102110.html


2019-09-05 13:02:43,476 connectionpool.py[396] DEBUG https://shouji.baidu.com:443 "GET /software/26102110.html HTTP/1.1" 200 None


iLight pro是基于蓝牙4.0的射频技术开发的手机应用，与彩灯配套使用。通过该应用与手机连接，只要点点手机，即可进行各种无线操控，您可将灯光设置成不同的颜色，再配合使用音乐，带您走进色彩斑斓的世界，体验您意想不到的情景，玩出我们的精彩，快快下载吧！iLight pro与彩灯连接成功后可实现以下功能：灯光控制：通过手机APP即可控制彩灯的开关、亮度、颜色、色温；炫彩灯效...iLight pro是基于蓝牙4.0的射频技术开发的手机应用，与彩灯配套使用。通过该应用与手机连接，只要点点手机，即可进行各种无线操控，您可将灯光设置成不同的颜色，再配合使用音乐，带您走进色彩斑斓的世界，体验您意想不到的情景，玩出我们的精彩，快快下载吧！iLight pro与彩灯连接成功后可实现以下功能：灯光控制：通过手机APP即可控制彩灯的开关、亮度、颜色、色温；炫彩灯效：手机内置多种炫彩灯效，满足你不同情境的需求；音乐律动：让灯光跟随着音乐起舞；多灯控制：配合新版彩灯时间多灯同时控制，同时播放歌曲，给你意想不到的惊喜体验；趣味：摇一摇，音乐控制，颜色变换，惊喜立现；定时关灯，助你好眠；定时开灯，灯和闹钟，一个都不能少；照片取色，情景换色；FM、音频输入，支持更多设备功能控制...更多趣味功能等你来发现！
dp=== 
match_name iLight pro
cat_list ['软件', '生活实用']
{'name': 'iLightpro', 'match_name': 'iLight pro', 'cat_list': '软件,生活实用', 'desc': 'iLight pro是基于蓝牙4.0的射频技术开发的手机应用，与彩灯配套使用。通过该应用与手机连接，只要点点手机，即可进行各种无线操控，您可将灯光设置成不同的颜色，再配合使用音乐，带您走进色彩斑斓的世界，体验您意想不到的情景，玩出我们的精彩，快快下载吧！iLight pro与彩灯连接成功后可实现以下功能：灯光控制：通过手机APP即可控制彩灯的开关、亮度、颜色、色温；炫彩灯效...iLight pro是基于蓝牙4.0的射频技术开发的手机应用，与彩灯配套使用。通过该应用与手机连接，只要点点手机，即可进行各种无线操控，您可将灯光设置成不同的颜色，再配合使用音乐，带您走进色彩斑斓的世界，体验您意想不到的情景，玩出我们的精彩，快快下

In [3]:
ph2  = get_ph2()
ph2.head()

,id,name,pkg,type_id
0,719a83f3e3ef4063b5365c2f776e7776,游戏王座,com.tencent.tmgp.yxwz,140203
1,ae8581eec07d4ccd803b8f4dda852b11,DMM GAMES,com.dmm.app.store,140206
2,6f717ad723294b9f87be597a5781125d,小钱袋,com.csxqd.xiaoqiandai,142106
3,667be1788d14476d9749c049ede87539,高人汇,com.ztm.providence,142701
4,c50569cc38414f19be6ce49bbe445e53,平行空間(原雙開大師),com.lbe.parallel.intl,140901


In [51]:

def get_ph2(fix=True):
    #df1 = pd.read_csv('./input/0823/final_apptype_train.dat', names=['name', 'pkg', 'type_id'], sep='\t')
    df1 = pd.read_csv('./input/0823/final_apptype_train.dat')

    # df1_index = get_train_ph2_index()
    # df1['id']=df1_index['id']

    df2 = pd.read_csv('./input/0823/appname_package.dat', names=['id', 'name', 'pkg'], sep='\t')

    ph2 = pd.concat([df1, df2])
    if fix!=True:
        return ph2
    else:
        ph2.name = ph2.name.astype(str)
        import re
        #ph2.name = ph2.name(lambda val: re.sub('(推荐)|(QQ浏览器)', '', val))
    
        ph2.pkg = ph2.pkg.fillna('').astype(str).str.lower()
    
        sp_name_list = ['444444444']
    
        ph2 = ph2.fillna('')
        ph2['name'] = ph2.apply(lambda row: ','.join([row.pkg, row['name']]) if len(row['name']) <= 0 or row['name'] in sp_name_list else row['name'], axis=1)#.astype(str)
    
        ph2['name'] = ph2.apply(lambda row: ','.join([row.pkg]) if row['name'] =='WXEntryActivity' else row['name'], axis=1)#.astype(str)
    
        ph2['name'] = ph2['name'].astype(str)
    
        ph2 = fix_name(ph2)
        ph2 = fix_pkg(ph2)
    
        return ph2.reset_index(drop=True)
    
    
ph2 = get_ph2(fix=False)
import re
reg = re.compile(r"""
\(
|\)
|（
|）
|\-
""" ,
re.VERBOSE
)

#ph2.loc[~pd.isna(ph2.name.apply(lambda val: reg.search(str(val))))]#.shape

ph2 = ph2.name.apply(lambda val: reg.split(str(val)))
ph2 = ph2.loc[ph2.apply(lambda val: len(val)>1)]
tmp = ph2.explode().to_frame().groupby('name').name.agg({'name_cnt':'count'}).sort_values('name_cnt')#.to_frame()
tmp['len_']=tmp.index.str.len()
tmp =  tmp.sort_values('len_')
tmp = tmp.loc[tmp.name_cnt>5]
tmp = tmp.loc[tmp.len_>2].sort_values('name_cnt')
#tmp.tail(30)
list(tmp.index)

/Users/lali2/dev/python/anaconda3/envs/tf14/lib/python3.6/site-packages/ipykernel_launcher.py:10: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  # Remove the CWD from sys.path while we load stuff.
/Users/lali2/dev/python/anaconda3/envs/tf14/lib/python3.6/site-packages/ipykernel_launcher.py:51: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version. Use                 named aggregation instead.

    >>> grouper.agg(name_1=func_1, name_2=func_2)



[' Android S2S',
 '美颜相机',
 '中国象棋',
 '资讯头条',
 '水印相机',
 '聊天交友',
 '鲸鱼阅读',
 '冠位指定',
 '宝宝超市',
 '腾讯视频',
 '信用管家',
 '跑男小镇',
 '手机管家',
 '微信双开',
 '拍拍贷借款',
 '爱奇艺',
 '大奖赛',
 '个人所得税',
 'Faceu激萌',
 '中国农业银行',
 'TapTap',
 '安全教育平台',
 '火影忍者 ',
 '讯飞输入法',
 ' 忍者大师',
 '清理大师',
 '手电筒',
 '经典版',
 '掌证宝',
 '驾校一点通',
 '二级建造师',
 '热血奥特曼',
 '愤怒的小鸟',
 '每日优鲜',
 '酷跑超人',
 '视频交友',
 '搜狗浏览器',
 '欢乐麻将',
 '微信多开',
 '水果连连看',
 '儿歌多多',
 '云闪付',
 '葫芦娃',
 '应用宝',
 '普通版',
 '咪咕圈圈',
 '儿童早教',
 '单机麻将',
 '中国移动',
 '雷ald',
 '手机淘宝',
 '每日瑜伽',
 '高清版',
 '我的世界',
 '暴击僵尸',
 '葫芦侠',
 'WiFi管家',
 '植物大战僵尸2',
 'CAD快速看图',
 'iVMS',
 '影视大全',
 '房天下',
 '免费小说',
 '一级建造师',
 'Fi 安全',
 'WiFi万能钥匙',
 '贷款借钱',
 '酷我音乐',
 '圣斗士星矢',
 '最新版',
 '中文版',
 '铃声多多',
 '交管12123',
 'QQ浏览器',
 '搜狗搜索加强版',
 'K歌小助手19',
 '兼职猫',
 '驾考宝典',
 '应用中心',
 '天气预报',
 '全民K歌',
 '倚天情缘',
 '儿童益智',
 '搜狗输入法',
 '单机版',
 '懒人听书',
 '酷狗音乐',
 '奥特曼',
 '双开助手',
 '开心版',
 '正版授权',
 '手机赚钱',
 '疯狂版',
 'Android',
 '下载中心',
 '血战到底',
 '恋舞OL',
 '正版3D回合手游',
 '欢乐斗地主',
 '疯狂原始人',
 '搜狗搜索',
 '四川麻将',
 '欢乐版',
 '锁屏精灵',
 '境·界',
 '儿童游戏',
 '魂之

In [ ]:
ph2.loc[]

['百度手机助手', '(', '奇迹暖暖', ')', '']

AttributeError: 'str' object has no attribute 'apply'

In [5]:
 

def get_data_from_bd(name):
    try:
        from requests.utils import quote
        name_new = quote(name)
        url2 = f'http://www.baidu.com/s?wd={name_new}%20安卓版%20安卓'
        print(url2)
        from lxml import html
        import requests
        #print(url2)
        response = requests.get(url2, timeout=5)
        #print(1)
        #print(response.text)
        tree = html.fromstring(response.content)
        # print(tree)
        # print(url2)
        # print(response.text)
        # desc = tree.xpath('/html/body/div[2]/div[2]/div[2]/div[1]/div[3]/div/div/text()[1]'
        # desc = tree.xpath('/html/body/div[2]/div[2]/div[2]/div[1]/div/div/div[1]/p/text()[1]')
        desc = tree.xpath('//*[@id="1" or @id="2"]//div[@class="c-abstract"]//text()')
        #print(2)
        desc = ''.join(desc)
        import re
        desc = re.sub('[0-9]*年[0-9]*月[0-9]*日.-.', '', desc)
        #print(3)
        res = {
            'name': name,
            #'match_name': name,
            'desc_bd': desc,
            'ct': pd.to_datetime('now'),

        }
    except Exception as e:
        print(name, url2)
        print(e)
        res = {'name': name, 'ct': pd.to_datetime('now'), }
    print(4)
    return res


get_data_from_bd('008彩票')

2019-09-01 15:31:53,498 connectionpool.py[206] DEBUG Starting new HTTP connection (1): www.baidu.com:80


http://www.baidu.com/s?wd=008%E5%BD%A9%E7%A5%A8%20安卓版%20安卓


2019-09-01 15:31:54,135 connectionpool.py[396] DEBUG http://www.baidu.com:80 "GET /s?wd=008%E5%BD%A9%E7%A5%A8%20%E5%AE%89%E5%8D%93%E7%89%88%20%E5%AE%89%E5%8D%93 HTTP/1.1" 200 None


4


{'name': '008彩票',
 'desc_bd': '811彩票下载app安卓版主流彩种多,覆盖人群广;APP开奖信息同步实时更新,帮助用户及时了解购彩资讯;进行各种购彩数据统计,提供专业的数据分析,811彩票下...大乐透008期历史同期号码汇总',
 'ct': Timestamp('2019-09-01 07:31:54.325893')}

In [5]:
df.loc[df.name != df.match_name]

,cat_list,closed_ids,ct_x,desc,dp,match_name,name,path,source,tag_list,wd_id,desc_bd,ct_y
0,"办公商务,笔记","6753331,6631597,6584885,6566707",1566920072.884119,清除干扰，刚刚好。 (OFFTIME) 统计并个性化 你的通讯，让你能专心于重要的事物。清除...,拔掉，就足够了！,断线时间OFFTIME,( OFFTIME ),NaN,NaN,"战斗 , 恐怖 , 阅读学习 , 常用工具 , 系统工具 , 电子书...",6518095,NaN,NaT
1,"通讯社交,电话通讯","6516344,6503396,6547028,280787",1566956853.496439,简单的应用程序显示在您的设备中的SIM卡的信息。它也显示保存在SIM卡上的联系人。 按选定的...,对于SIM卡上的联系人进行增删改查等操作,SIM卡,(U)SIM卡应用,NaN,NaN,,2230555,NaN,NaT
2,"系统工具,安全","7949692,566489,6618765,596157",1566956856.41078,★QQ安全中心，用心守护您的QQ★当您的帐号出现安全风险时；当您忘记了QQ密码时；当您消费Q...,有了它，再也不用担心QQ，微信，游戏账号被黑被盗。,QQ安全中心,(推荐)安全安装,NaN,NaN,"安全工具 , 腾讯 , 系统安全 , 密码",280221,NaN,NaT
3,"系统工具,优化","6597626,827557,138133,6767113",1566956860.0010839,现在短信多为通知类短信，多而杂，批量删除又担心误删，不删又杂乱无章？有用短信找寻困难，清理垃...,专注有用短信归纳,短信夹,0 类信息,NaN,NaN,,7600827,NaN,NaT
4,"通讯社交,聊天,电话通讯","6597626,2316991,138133,34555",1566956861.4199538,GO短信加强版，全球最受用户欢迎的实用手机短信应用，加强系统短信锁不具备的众多功能。主题换肤...,安全锁、私密箱、骚扰拦截，短信从此更安全,GO短信,0 类短信,NaN,NaN,"娱乐 , 小清新 , 文字 , 涂鸦 , 表情 , 备份 , 通...",280253,NaN,NaT
5,"经营策略,养成","8015792,7879821,7817390,7932155",1566956865.133682,“救命！！！”某年某月某日，你又收到了一条求救信息……尬聊王，啊不对，mini讯息是一款流程...,一款模拟短信收发进行的游戏,mini讯息,0 類訊息,NaN,NaN,,8045128,NaN,NaT
7,"旅游出行,用车租车","7915635,7593502,7601800,6522898",1566956869.932728,"软件描述:快的打车司机版, 全新登场!学会只需 1 分钟, 每月多赚 2000 元!全国覆盖...",智能的打车信息推送方式，让你不错过每一笔生意,快的司机,007司机,NaN,NaN,"驾驶 , 技巧 , 极速 , 出行 , 打车 , 租车",50995,NaN,NaT
9,"金融理财,彩票","7491223,6695847,7840670,8037903",1566956874.819743,500APP，覆盖全网热门彩种，全方位为您服务。全网公告、走势图表、资讯预测等数据服务；足篮...,体彩玩家主场，大奖诞生之地！,500彩票,008彩票,NaN,NaN,"生活应用 , 网上购物",40975,NaN,NaT
10,"新闻阅读,新闻资讯","6604794,5789169,282275,6194542",1566956879.7078662,你的一点更好看！ 【个性一点】高度私人订制，妙不可言！你是关注美容变白瘦身穿衣搭配的小...,内容还可以，不过新闻软件都有些雷同。,一点资讯,008资讯版,NaN,NaN,,1236671,NaN,NaT
11,"休闲益智,解谜","6630928,6651919,6608405,7883081",1566956881.371834,《校园 SCHOOL》一款恐怖惊悚类逃脱游戏，被阴郁的空气所包围的废校，是主角这次参加试胆大...,一款恐怖惊悚类逃脱游戏。,校园,00校园,NaN,NaN,"角色扮演 , 免谷歌验证 , 逃脱 , 恐怖 , 解谜",6292471,NaN,NaT


In [7]:
df1 = pd.read_csv('./input/0823/final_apptype_train.dat', names=['name', 'pkg', 'type_id'], sep='\t')
df2 = pd.read_csv('./input/0823/appname_package.dat', names=['id', 'name', 'pkg'], sep='\t')

df = pd.concat([df1, df2])[['name', 'pkg']]
df.head()

/Users/lali2/dev/python/anaconda3/envs/tf14/lib/python3.6/site-packages/ipykernel_launcher.py:4: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  after removing the cwd from sys.path.


,name,pkg
0,游戏王座,com.tencent.tmgp.yxwz
1,DMM GAMES,com.dmm.app.store
2,小钱袋,com.csxqd.xiaoqiandai
3,高人汇,com.ztm.providence
4,平行空間(原雙開大師),com.lbe.parallel.intl


In [35]:

def get_data_by_pkg(pkg):
    try:
        from lxml import html
        import requests

        url = f'http://app.mi.com/details?id={pkg}'
        response = requests.get(url)
        tree = html.fromstring(response.content)


        desc = tree.xpath('/html/body/div[6]/div[1]/div[4]/p/text()')
        desc = ''.join(desc) if len(desc) > 0 else ''
        desc = desc.replace('\t', '')
        
        cat_list = tree.xpath('/html/body/div[6]/div[1]/div[2]/div[1]/div/p[2]/text()[1]')
        
        app_name = tree.xpath('/html/body/div[6]/div[1]/div[2]/div[1]/div/h3/text()')[0]
        print(cat_list,  app_name,  desc)
        
        res = {
            'pkg':pkg,
            'name': app_name,
            'cat_list': ','.join(cat_list),      
            'desc': desc,             
            'ct': pd.to_datetime('now'),
            'source': 'xm'
        }
 
    except Exception as e:
        print(e, url)
        res = { 'pkg': pkg, 'source': 'xm'  }
    return res
todo = get_todo_list_pkg()
for item in todo[:50]:
    print(get_data_by_pkg(item))

merge file in  fold to :./output/spider/wdj/merge_76673.h5


/Users/lali2/dev/python/anaconda3/envs/tf14/lib/python3.6/site-packages/ipykernel_launcher.py:8: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  
2019-08-30 18:39:33,873 connectionpool.py[206] DEBUG Starting new HTTP connection (1): app.mi.com:80
2019-08-30 18:39:35,536 connectionpool.py[396] DEBUG http://app.mi.com:80 "GET /details?id=1234.app HTTP/1.1" 302 56
2019-08-30 18:39:35,602 connectionpool.py[396] DEBUG http://app.mi.com:80 "GET / HTTP/1.1" 200 None
2019-08-30 18:39:35,617 connectionpool.py[206] DEBUG Starting new HTTP connection (1): app.mi.com:80
2019-08-30 18:39:35,742 connectionpool.py[396] DEBUG http://app.mi.com:80 "GET /details?id=3z62kjgd4f165851.app HTTP/1.1" 302 56
2019-08-30 18:39:35,805 connectionpool.py[396] DEBUG http://app.mi.com:80 "GET / HTTP/1.1" 2

list index out of range http://app.mi.com/details?id=1234.app
{'pkg': '1234.app', 'source': 'xm'}


2019-08-30 18:39:35,823 connectionpool.py[206] DEBUG Starting new HTTP connection (1): app.mi.com:80
2019-08-30 18:39:35,902 connectionpool.py[396] DEBUG http://app.mi.com:80 "GET /details?id=91pandareaderhd.app HTTP/1.1" 302 56
2019-08-30 18:39:35,960 connectionpool.py[396] DEBUG http://app.mi.com:80 "GET / HTTP/1.1" 200 None
2019-08-30 18:39:35,975 connectionpool.py[206] DEBUG Starting new HTTP connection (1): app.mi.com:80


list index out of range http://app.mi.com/details?id=3z62kjgd4f165851.app
{'pkg': '3z62kjgd4f165851.app', 'source': 'xm'}
list index out of range http://app.mi.com/details?id=91pandareaderhd.app
{'pkg': '91pandareaderhd.app', 'source': 'xm'}


...

list index out of range http://app.mi.com/details?id=abc.mlive
{'pkg': 'abc.mlive', 'source': 'xm'}
list index out of range http://app.mi.com/details?id=ac.xzreader.book.reader
{'pkg': 'ac.xzreader.book.reader', 'source': 'xm'}


2019-08-30 18:40:05,426 connectionpool.py[396] DEBUG http://app.mi.com:80 "GET /details?id=acr.browser.barebones HTTP/1.1" 302 56
2019-08-30 18:40:05,546 connectionpool.py[396] DEBUG http://app.mi.com:80 "GET / HTTP/1.1" 200 None
2019-08-30 18:40:05,556 connectionpool.py[206] DEBUG Starting new HTTP connection (1): app.mi.com:80
2019-08-30 18:40:05,625 connectionpool.py[396] DEBUG http://app.mi.com:80 "GET /details?id=activity.app.bosshive.bosssoft.com.fzhmcx HTTP/1.1" 302 56
2019-08-30 18:40:05,698 connectionpool.py[396] DEBUG http://app.mi.com:80 "GET / HTTP/1.1" 200 None
2019-08-30 18:40:05,709 connectionpool.py[206] DEBUG Starting new HTTP connection (1): app.mi.com:80


list index out of range http://app.mi.com/details?id=acr.browser.barebones
{'pkg': 'acr.browser.barebones', 'source': 'xm'}
list index out of range http://app.mi.com/details?id=activity.app.bosshive.bosssoft.com.fzhmcx
{'pkg': 'activity.app.bosshive.bosssoft.com.fzhmcx', 'source': 'xm'}


2019-08-30 18:40:05,789 connectionpool.py[396] DEBUG http://app.mi.com:80 "GET /details?id=activty.xiaoi.com.xfdemo HTTP/1.1" 302 56
2019-08-30 18:40:05,878 connectionpool.py[396] DEBUG http://app.mi.com:80 "GET / HTTP/1.1" 200 None
2019-08-30 18:40:05,894 connectionpool.py[206] DEBUG Starting new HTTP connection (1): app.mi.com:80
2019-08-30 18:40:05,980 connectionpool.py[396] DEBUG http://app.mi.com:80 "GET /details?id=actpiano.liuxqsmile.android HTTP/1.1" 302 56
2019-08-30 18:40:06,030 connectionpool.py[396] DEBUG http://app.mi.com:80 "GET / HTTP/1.1" 200 None


list index out of range http://app.mi.com/details?id=activty.xiaoi.com.xfdemo
{'pkg': 'activty.xiaoi.com.xfdemo', 'source': 'xm'}
list index out of range http://app.mi.com/details?id=actpiano.liuxqsmile.android
{'pkg': 'actpiano.liuxqsmile.android', 'source': 'xm'}


In [2]:
df = merge_file('./output/spider/xm')
df.head()

2019-08-30 19:12:45,523 utils.py[141] INFO NumExpr defaulting to 8 threads.


merge file in  fold to :./output/spider/xm/merge_8500.h5


/Users/lali2/Documents/workspace_py/xf_tag/spider/mi.py:283: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  df = pd.concat([pd.read_hdf(file, 'wdj') for file in glob(f'{fold}/*.h5')])
/Users/lali2/dev/python/anaconda3/envs/tf14/lib/python3.6/site-packages/pandas/core/generic.py:2531: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block0_values] [items->['cat_list', 'ct', 'desc', 'name', 'pkg', 'source']]

  pytables.to_hdf(path_or_buf, key, self, **kwargs)


,cat_list,ct,desc,name,pkg,source
0,NaN,2019-08-30 10:53:35.123543,NaN,NaN,com.djsds,xm
0,NaN,2019-08-30 10:53:35.580168,NaN,NaN,com.jpush.hisenseengineerapp.crm,xm
1,NaN,2019-08-30 10:53:35.644032,NaN,NaN,com.djt.personreadbean,xm
2,NaN,2019-08-30 10:53:36.191796,NaN,NaN,com.djtest,xm
0,实用工具,2019-08-30 10:53:36.379366,验钞灯超亮手电，验钞灯功能强大方便快捷。\r特色功能\r验钞灯快速启动：轻轻一点即可打开照明...,防身手电,com.wolfer.flashlight,xm


In [3]:
df.sort_values('desc')

,cat_list,ct,desc,name,pkg,source
759,金融理财,2019-08-30 11:01:59.552999,"""AI考拉（原考拉理财）由广州财略金融信息科技有限公司打造，是华南地区具有影响力的互联网金融...",AI考拉,com.kalengo.loan,xm
1278,实用工具,2019-08-30 11:07:07.189773,"""风筝守护”是一款针对家长管理孩子或者子女帮助父母使用手机及保护家人安全的APP，您可以使用...",风筝守护,com.kiteguard,xm
1056,居家生活,2019-08-30 11:04:24.333542,#为餐厅商户提供简洁、专业的滴滴平台服务#\r\r- 海量订单，合理推荐，没有难做的生意\r...,滴滴商户,com.xiaojukeji.didi.soda.merchant,xm
1097,实用工具,2019-08-30 11:05:08.256189,*** 一个华人喜爱的翻译软件 ***\r*** 累计下载量已达350万次 ***\r[中国...,特快翻译,com.rockcatstudio,xm
944,学习教育,2019-08-30 11:03:52.904062,*为3-6岁儿童设计的数学启蒙课程\r*在快乐的冒险游戏中理解数学概念\r*和米亚米娜一起，...,数学王国,cc.morechinese.MathKingdom,xm
1468,实用工具,2019-08-30 11:08:51.170361,*因加密性能限制，抱歉不支持无指纹识别的老机型\r\r密码键盘——安全便捷的密码管理器\r帮...,密码键盘,com.kongzue.secretinput,xm
1020,学习教育,2019-08-30 11:04:35.550758,"*是否曾经想说英语,却不知道如何开口?\r*是否曾经想找一份特殊情景说特殊语言的资料?\r*...",英语口语听说练,chengqiang.celever2005.English8900,xm
1266,时尚购物,2019-08-30 11:06:37.379944,----大润发的网上生鲜超市----\r1. 新人礼包：注册即送50元券\r2.下单后zui...,大润发优鲜,com.rt.market.fresh,xm
146,学习教育,2019-08-30 10:56:09.310445,0至7岁宝宝都喜欢的音乐游戏\r1、小木琴、钢琴、电子琴、架子鼓，男孩女孩都喜欢；\r2、播...,宝宝儿歌大全,com.doding.childrenmusic,xm
1010,效率办公,2019-08-30 11:04:38.405414,1. 强大的时间记录功能， 目标、考试、约会、游玩、假期、还款日， 全部望在眼里。 通过倒计...,计时人生,com.khsoft.lifetimer,xm


In [6]:
def get_data_from_bd(name):
    try:
        url2 = f'http://www.baidu.com/s?wd={name}%20安卓版'
        from lxml import html
        import requests
        from requests.utils import quote
        print(0)
        response = requests.get(url2)
        print(1)
        # print(response.text)
        tree = html.fromstring(response.content)
        # print(tree)
        # print(url2)
        # print(response.text)
        # desc = tree.xpath('/html/body/div[2]/div[2]/div[2]/div[1]/div[3]/div/div/text()[1]'
        # desc = tree.xpath('/html/body/div[2]/div[2]/div[2]/div[1]/div/div/div[1]/p/text()[1]')
        desc = tree.xpath('//*[@id="1" or @id="2"]//div[@class="c-abstract"]//text()')
        print(2)
        desc = ''.join(desc)
        import re
        desc = re.sub('[0-9]*年[0-9]*月[0-9]*日.-.', '', desc)
        print(3)
        res = {
            'name': name,
            'desc_bd': desc,
            'ct': pd.to_datetime('now'),

        }
    except Exception as e:
        print(name, url2)
        print(e)
        res = {'name': name, 'ct': pd.to_datetime('now'), }
    print(4)
    return res





get_data_from_bd('店铺通')
#get_data_from_wdj('买车宝')

2019-08-30 15:01:50,172 connectionpool.py[206] DEBUG Starting new HTTP connection (1): www.baidu.com:80


0


2019-08-30 15:01:50,671 connectionpool.py[396] DEBUG http://www.baidu.com:80 "GET /s?wd=%E5%BA%97%E9%93%BA%E9%80%9A%20%E5%AE%89%E5%8D%93%E7%89%88 HTTP/1.1" 200 None


1
2
3
4


{'name': '店铺通',
 'desc_bd': '绿色资源网收集的店铺通安卓版手机版本是一款门店管理服务手机软件,支持在手机上查看详细的门店库存和销售业绩,支持对订单和客户进行管理,欢迎有需要的朋友到绿色资源...PP助手为您提供最新店铺通下载,店铺通安卓手机版免费下载,PP助手让你尽享好玩的手机应用',
 'ct': Timestamp('2019-08-30 07:01:50.881553')}

In [17]:
from file_cache.cache import file_cache
@file_cache()
def get_final():
    wdj = pd.concat([pd.read_hdf(file, 'wdj') for file in glob('./output/spider/wdj/*.h5')])
    wdj.ct = wdj.ct.astype('str')
    wdj = wdj.sort_values('ct').drop_duplicates('name', keep='last')
    bd = pd.concat([pd.read_hdf(file, 'wdj') for file in glob('./output/spider/bd/*.h5')])
    bd = bd.sort_values('ct').drop_duplicates('name', keep='last')
    
    return pd.merge(wdj, bd, how='left', on='name')

df =  get_final()
df.head()

2019-08-30 13:45:25,606 cache.py[40] DEBUG Can not find cache from file:./cache/get_final==.pickle
/Users/lali2/dev/python/anaconda3/envs/tf14/lib/python3.6/site-packages/ipykernel_launcher.py:4: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  after removing the cwd from sys.path.


,cat_list,closed_ids,ct_x,desc,dp,match_name,name,path,source,tag_list,wd_id,desc_bd,ct_y
0,"办公商务,笔记","6753331,6631597,6584885,6566707",1566920072.884119,清除干扰，刚刚好。 (OFFTIME) 统计并个性化 你的通讯，让你能专心于重要的事物。清除...,拔掉，就足够了！,断线时间OFFTIME,( OFFTIME ),NaN,NaN,"战斗 , 恐怖 , 阅读学习 , 常用工具 , 系统工具 , 电子书...",6518095,NaN,NaT
1,"通讯社交,电话通讯","6516344,6503396,6547028,280787",1566956853.496439,简单的应用程序显示在您的设备中的SIM卡的信息。它也显示保存在SIM卡上的联系人。 按选定的...,对于SIM卡上的联系人进行增删改查等操作,SIM卡,(U)SIM卡应用,NaN,NaN,,2230555,NaN,NaT
2,"系统工具,安全","7949692,566489,6618765,596157",1566956856.41078,★QQ安全中心，用心守护您的QQ★当您的帐号出现安全风险时；当您忘记了QQ密码时；当您消费Q...,有了它，再也不用担心QQ，微信，游戏账号被黑被盗。,QQ安全中心,(推荐)安全安装,NaN,NaN,"安全工具 , 腾讯 , 系统安全 , 密码",280221,NaN,NaT
3,"系统工具,优化","6597626,827557,138133,6767113",1566956860.0010839,现在短信多为通知类短信，多而杂，批量删除又担心误删，不删又杂乱无章？有用短信找寻困难，清理垃...,专注有用短信归纳,短信夹,0 类信息,NaN,NaN,,7600827,NaN,NaT
4,"通讯社交,聊天,电话通讯","6597626,2316991,138133,34555",1566956861.4199538,GO短信加强版，全球最受用户欢迎的实用手机短信应用，加强系统短信锁不具备的众多功能。主题换肤...,安全锁、私密箱、骚扰拦截，短信从此更安全,GO短信,0 类短信,NaN,NaN,"娱乐 , 小清新 , 文字 , 涂鸦 , 表情 , 备份 , 通...",280253,NaN,NaT


In [25]:
df.loc[(df.desc_bd.str.len()>1 ) & (df.desc_bd.str.len()<50 )]

,cat_list,closed_ids,ct_x,desc,dp,match_name,name,path,source,tag_list,wd_id,desc_bd,ct_y
10277,"金融理财,银行","597421,86961,43089,54313",2019-08-29 03:26:56.941833,,No desc,上海银行,上海银行,NaN,wdj,,6516136,"PP助手为您提供最新上海银行下载,上海银行安卓手机版免费下载,PP助手让你尽享好玩的手机应用",2019-08-30 05:28:42.363773


In [26]:
get_data_from_bd('Bumble Knife')

2019-08-30 14:09:31,232 connectionpool.py[206] DEBUG Starting new HTTP connection (1): www.baidu.com:80
2019-08-30 14:09:31,723 connectionpool.py[396] DEBUG http://www.baidu.com:80 "GET /s?wd=Bumble%20Knife%20%E5%AE%89%E5%8D%93 HTTP/1.1" 200 None


{'name': 'Bumble Knife',
 'desc_bd': 'PP助手为您提供最新Bumble Knife下载,Bumble Knife安卓手机版免费下载,PP助手让你尽享好玩的手机应用Time your actions, target carefully and become the Bumble Knife master !  开发者:互联网 安卓版下载 苹果版下载 预订激活码 订阅游戏 ...',
 'ct': Timestamp('2019-08-30 06:09:32.151528')}

In [12]:
bd.groupby('name').name.count()

name
#ffababab       1
13078971364     1
13524529922     1
13764409917     1
15021285639     1
               ..
時光小屋            1
時計              3
時鐘             11
晃咖              1
晃晃麻将            1
Name: name, Length: 2113, dtype: int64

In [5]:
#url2 = 'https://www.wandoujia.com/apps/1509817'
#url2 ='https://www.wandoujia.com/apps/7894236'
#url2 ='https://www.wandoujia.com/apps/7596952'
#url2 = 'https://www.wandoujia.com/apps/7970270'
url2 = 'https://www.wandoujia.com/apps/6586136'
url2 = 'https://www.wandoujia.com/apps/280379'
url2 = 'https://www.wandoujia.com/apps/280367'
url2 = 'https://www.wandoujia.com/apps/7837548'

from lxml import html
import requests
from requests.utils import quote
response = requests.get(url2)
tree = html.fromstring(response.content)
#desc = tree.xpath('/html/body/div[2]/div[2]/div[2]/div[1]/div[3]/div/div/text()[1]'
#desc = tree.xpath('/html/body/div[2]/div[2]/div[2]/div[1]/div/div/div[1]/p/text()[1]')
desc = tree.xpath('/html/body/div[2]/div[2]/div[2]/div[1]/div/div/div[1]/*/text()')
desc = tree.xpath('/html/body/div[2]/div[2]/div[2]/div/div/div/*/text()')
desc = tree.xpath('/html/body/div[2]/div[2]/div[2]/div/*/div[@itemprop="description"]//text()')#[0]
desc = tree.xpath('/html/body/div[2]/div[2]/div/div/*/div[@itemprop="description"]//text()')

dp = tree.xpath('/html/body/div[2]/div[2]/div[2]//div[@class="editorComment"]/div[@class="con"]/text()')  # [0]
dp = dp[0] if len(dp) > 0 else 'No desc'
print('dp', dp)

print('desc', desc)
# print(dir(desc))
# print(desc)
# print('hello')

# desc = ''.join(desc) if len(desc) > 0 else 'No desc'
# desc = desc.replace('\t','')
# desc = desc.replace('\r','')
# desc = desc.replace('\n','')
# print(desc)
# #html = f'http://app.mi.com/details?id={pkg}'

2019-08-30 22:40:56,516 connectionpool.py[815] DEBUG Starting new HTTPS connection (1): www.wandoujia.com:443
2019-08-30 22:40:57,287 connectionpool.py[396] DEBUG https://www.wandoujia.com:443 "GET /apps/7837548 HTTP/1.1" 200 None


dp No desc
desc ['在宇宙诞生之前，是一片虚空。在这虚空之中，无数星系正在萌生着自主的意识，在数千万年的混沌期中，他们逐渐诞生了自己守护神，彼此之间进行着无休止的永世斗争。 游戏特色： 五分钟一局 休闲外壳硬核竞技 无论3V3团队竞技，或是六人混战荒岛求生，在碎片化的时间里可以随时随时来上一局紧张刺激的对战！游戏规则简单上手，将敌人推到界外即是一切！但要想驾驭战场成为高手，各种小技巧和心理博弈会让你惊呼不已！ 特色英雄集结 全新Moba体验 告别了千篇一律的寒冰、盖，每个星座守卫都有自己独一无二的特色，充分享受角色使用技巧和魅力，有多少英雄就有多少种获胜方式，去发掘，去探寻属于自己的Style，也许下个英雄就是你的本命！ 公平竞技 宝物百变搭配 舍弃了直接加属性的设计，而是用更加考验时机和操作的宝物来创建玩家自己独特的角色，玩家不仅需要根据英雄技能，更要依照自己对于英雄的理解，是让英雄变成草丛伏地魔？或是变成莽汉一波？又或者濒血反戈一击？即便是同一个英雄也能完全有不同的玩法，全凭你的选择！ 创新多样玩法 开黑单排随你挑选 3V3的团队竞技更侧重英雄之间的技能搭配和释放顺序，以及适当时机自杀曲线救国？六人荒岛生存混战，则是一场老阴逼们的互阴大会，二桃杀三士、螳螂捕蝉黄雀在后、三十六计走为上计，各种见招拆招的策略和极限操作让人血脉喷张。而颇有拳皇风格的1v1擂台赛，则既能满足让大神之间高手过招，也能单挑全队享受装逼如风的快感，带妹神器！']


In [3]:

def get_data_from_wdj(name):
    try:
        #print('=' * 20)
        from lxml import html
        import requests
        from requests.utils import quote
        url, url2 = None,  None
        name_new = quote(name)
        url = f'https://www.wandoujia.com/search?key={name_new}&source=index'
        url2 = None
        response = requests.get(url)
        tree = html.fromstring(response.content)
        link = tree.xpath('//*[@id="j-search-list"]/li[2]/a')[0]
        url2 = link.get("href")
        #print(url2)
        response = requests.get(url2)
        tree = html.fromstring(response.content)

        desc = tree.xpath('/html/body/div[2]/div[2]/div[2]/div/*/div[@itemprop="description"]//text()')
        desc = ''.join(desc) if len(desc) > 0 else ''
        desc = desc.replace('\t', '')
        desc = desc.replace('\r', '')
        desc = desc.replace('\n', '').strip()

        dp = tree.xpath('/html/body/div[2]/div[2]/div[2]//div[@class="editorComment"]/div[@class="con"]/text()')
        dp = dp[0] if len(dp) > 0 else ''
        dp = dp.strip()

        closed_ids = tree.xpath('/html/body/div[2]/div[2]/div[2]/div[2]/div[3]/ol/li[*]/a[1]')  # [0]

        match_name = tree.xpath('/html/body/div[2]/div[2]/div[1]//div/p/span[@itemprop="name"]/text()')[0]

        cat_list = tree.xpath('/html/body/div[2]/div[2]/div[2]/div[2]/div[1]/dl/dd[2]/a/text()')
        tag_list = tree.xpath('/html/body/div[2]/div[2]/div[2]/div[2]/div[1]/dl/dd[3]/div/div/a/text()')
        closed_ids = [item.get('href').split('/')[-1] for item in closed_ids]
        path = tree.xpath('/html/body/div[2]/div[1]/div[2]//span[@itemprop="title"]/text()')[0]

        wd_id = url2.split('/')[-1]
        res = {
            'wd_id': wd_id,
            'name': name,
            'match_name': str(match_name),
            'tag_list': ','.join(tag_list),
            'cat_list': ','.join(cat_list),
            'closed_ids': ','.join(closed_ids),
            'path':     str(path),
            'desc':     desc,
            'dp': dp.strip().replace('\t', ''),  # 点评
            'ct':pd.to_datetime('now'),
            'source':'wdj'

        }

        #print(res)

    except Exception as e:
        print(name)
        print(name, url2 or url)
        print(e)
        res = {'name': name,'ct':pd.to_datetime('now'), }

        # if url2 is None:
        #     res = get_data_from_bd(name)

    return res

get_data_from_wdj('新浪新闻')

2019-08-30 12:33:23,831 connectionpool.py[815] DEBUG Starting new HTTPS connection (1): www.wandoujia.com:443
2019-08-30 12:33:24,745 connectionpool.py[396] DEBUG https://www.wandoujia.com:443 "GET /search?key=%E6%96%B0%E6%B5%AA%E6%96%B0%E9%97%BB&source=index HTTP/1.1" 302 92920
2019-08-30 12:33:25,402 connectionpool.py[396] DEBUG https://www.wandoujia.com:443 "GET /search/13191518958669166218 HTTP/1.1" 200 None
2019-08-30 12:33:25,535 connectionpool.py[815] DEBUG Starting new HTTPS connection (1): www.wandoujia.com:443
2019-08-30 12:33:25,993 connectionpool.py[396] DEBUG https://www.wandoujia.com:443 "GET /apps/280367 HTTP/1.1" 200 None


{'wd_id': '280367',
 'name': '新浪新闻',
 'match_name': '新浪新闻',
 'tag_list': '  益智休闲  ,  新闻资讯  ,  阅读  ,  资讯  ,  剧情游戏  ,  新闻  ,  订阅  ,  门户  ',
 'cat_list': '新闻阅读,新闻资讯',
 'closed_ids': '306653,7656564,1466693,280347',
 'path': '软件分类',
 'desc': '新浪新闻客户端，由新浪官方出品，第一时间获取全球新闻资讯，国内国外要闻，精彩的体育赛事报道，金融财经动向，影视事件，还有独家微博“微”新闻，精彩随你看，新闻、星座、笑话一个都不少。* 新闻: 每天更新海量资讯内容，最快最全的头条要闻。第一时间获悉国内外时事，体育赛事报道，金融财经动向，影视事件，社会、军事、科技新闻资讯。 * 热门：实时热点新闻，热门话题排行，快速了解当日最热事件。* 订阅: 自定义频道内容和顺序，定制属于自己的新闻。 * 手势: 流畅的手势操作，左右滑动切换页面，享受智能便捷服务。 * 直播: 中超、亚冠，欧冠、英超、世界杯精彩足球赛事陪你度过日夜，还有独家NBA精彩赛事以及特邀嘉宾解说，图文视频形式丰富，实时关注最新赛况。 * 高清图片: 新闻热点、八卦，高清美图一键分享，每天数百张更新，天天都有惊喜！ * 免密登录 :手机验证码一键登录，再也不用担心忘记密码啦。* 绑定微博 :一键绑定微博帐号，随时随地分享新闻。* 分享：微信、QQ空间、新浪微博等随意分享，时事、大事、趣事先人一步。* 接收推送: 精品内容、重大事件，第一时间获取实时资讯。 * 消息盒子：不让你错过任何一条精彩push内容* 收藏: 收藏感兴趣的内容，稍后再读。 * 夜间模式: 夜间阅读，拒绝刺眼，舒适体验。 * 评论: 针对新闻事件发表个人观点或吐槽,回复评论与其他用户互动。 * 离线阅读: 离线下载新闻内容，上班途中即知天下事。【联系我们】欢迎加入新浪新闻用户反馈群，帮助我们改进应用问题！QQ群号：213085440这里可以随时吐槽、分享、评论~还有精美礼品等你来拿哦^_^  新浪新闻2019更新内容「精选功能上线」-二楼搬迁至底部导航精选模块-通过精品栏目可以探索更多有趣的内容-精品栏目增加关注功能「体

In [5]:
set(['a','b','c']) - set(['d','b','c'] )

{'a'}

In [79]:

def get_data_from_wdj(name):
    try:
        #print('=' * 20)
        from lxml import html
        import requests
        from requests.utils import quote
        name_new = quote(name)
        url = f'https://www.wandoujia.com/search?key={name_new}&source=index'
        url2 = None
        response = requests.get(url)
        tree = html.fromstring(response.content)
        link = tree.xpath('//*[@id="j-search-list"]/li[2]/a')[0]
        url2 = link.get("href")
        #print(url2)
        response = requests.get(url2)
        tree = html.fromstring(response.content)

        desc = tree.xpath('/html/body/div[2]/div[2]/div[2]/div/*/div[@itemprop="description"]//text()')
        desc = ''.join(desc) if len(desc) > 0 else 'No desc'
        desc = desc.replace('\t', '')
        desc = desc.replace('\r', '')
        desc = desc.replace('\n', '')

        dp = tree.xpath('/html/body/div[2]/div[2]/div[2]//div[@class="editorComment"]/div[@class="con"]/text()')
        dp = dp[0] if len(dp) > 0 else 'No desc'
        print(1)
        closed_ids = tree.xpath('/html/body/div[2]/div[2]/div[2]/div[2]/div[3]/ol/li[*]/a[1]')  # [0]
        print(2)
        match_name = tree.xpath('/html/body/div[2]/div[2]/div[1]//div/p/span[@itemprop="name"]/text()')
        print(3)
        cat_list = tree.xpath('/html/body/div[2]/div[2]/div[2]/div[2]/div[1]/dl/dd[2]/a/text()')
        tag_list = tree.xpath('/html/body/div[2]/div[2]/div[2]/div[2]/div[1]/dl/dd[3]/div/div/a/text()')
        path = tree.xpath('/html/body/div[2]/div[1]/div[2]//span[@itemprop="title"]/text()')[0]
        closed_ids = [item.get('href').split('/')[-1] for item in closed_ids]

        wd_id = url2.split('/')[-1]
        res = {
            'wd_id': wd_id,
            'name': name,
            'match_name': match_name,
            'tag_list': ','.join(tag_list),
            'path': path,
            'cat_list': ','.join(cat_list),
            'closed_ids': ','.join(closed_ids),
            'desc': desc,
            'dp': dp.strip().replace('\t', ''),  # 点评
            'ct':pd.to_datetime('now'),
            'source':'wdj'

        }

        #print(res)

    except Exception as e:
        print(name, url2 or url)
        print(e)
        res = {'name': name,'ct':pd.to_datetime('now'), }
    return res


get_data_from_wdj('失控飞车')

2019-08-29 13:22:03,932 connectionpool.py[815] DEBUG Starting new HTTPS connection (1): www.wandoujia.com:443
2019-08-29 13:22:04,908 connectionpool.py[396] DEBUG https://www.wandoujia.com:443 "GET /search?key=%E5%A4%B1%E6%8E%A7%E9%A3%9E%E8%BD%A6&source=index HTTP/1.1" 200 None
2019-08-29 13:22:04,943 connectionpool.py[815] DEBUG Starting new HTTPS connection (1): www.wandoujia.com:443
2019-08-29 13:22:05,468 connectionpool.py[396] DEBUG https://www.wandoujia.com:443 "GET /apps/7899456 HTTP/1.1" 200 None


1
2
3


{'wd_id': '7899456',
 'name': '失控飞车',
 'match_name': ['失控飞车'],
 'tag_list': '',
 'path': '休闲益智',
 'cat_list': '',
 'closed_ids': '',
 'desc': '《失控飞车》是一款将赛车与跑酷合二为一创新型玩法、Q萌搞怪的美术风格、超高质量的游戏体验的跑酷手游大作。卡通人物驾驶汽车从山上往下行驶，但是刹车失灵了，只能一路向下行驶，通过无尽的下坡躲避障碍物。通过游戏不仅可以收藏上百种精美车模，快来开启你的精彩刺激的卡通竞速之旅吧！',
 'dp': 'No desc',
 'ct': Timestamp('2019-08-29 05:22:05.481636'),
 'source': 'wdj'}

In [219]:

def get_data_from_bd(name):
    url2 = f'http://www.baidu.com/s?wd={name}%20安卓'
    from lxml import html
    import requests
    from requests.utils import quote

    response = requests.get(url2)
    # print(response.text)
    tree = html.fromstring(response.content)
    # print(tree)
    # print(url2)
    # print(response.text)
    # desc = tree.xpath('/html/body/div[2]/div[2]/div[2]/div[1]/div[3]/div/div/text()[1]'
    # desc = tree.xpath('/html/body/div[2]/div[2]/div[2]/div[1]/div/div/div[1]/p/text()[1]')
    desc = tree.xpath('//*[@id="1" or @id="2"]//div[@class="c-abstract"]//text()')

    desc = ''.join(desc)
    import re
    desc = re.sub('[0-9]*年[0-9]*月[0-9]*日.-.', '', desc)

    res = {
        'name': name,
        'desc': desc,
        'ct': pd.to_datetime('now'),
        'source': 'bd'

    }
    return res

get_data_from_bd('PAYCO')

2019-08-29 18:49:44,691 connectionpool.py[206] DEBUG Starting new HTTP connection (1): www.baidu.com:80
2019-08-29 18:49:45,513 connectionpool.py[396] DEBUG http://www.baidu.com:80 "GET /s?wd=PAYCO%20%E5%AE%89%E5%8D%93 HTTP/1.1" 200 None


{'name': 'PAYCO',
 'desc': '4天前\xa0-\xa0安卓手机应用> apk>PAYCO PAYCO综述 历史版本 扫描二维码,下载应用至手机 应用价格: 应用大小:19.81MB 推荐指数:50 安全检测:安全 版本: 2.3.7 更新: ...4天前\xa0-\xa0安卓手机应用> apk>PAYCO PAYCO综述 历史版本 扫描二维码,下载应用至手机 应用价格: 应用大小:32.1MB 推荐指数:50 安全检测:安全 版本: 2.4.11 更新: ...',
 'ct': Timestamp('2019-08-29 10:49:45.753485'),
 'source': 'bd'}

In [315]:
#url2 = 'https://www.wandoujia.com/apps/1509817'
#url2 ='https://www.wandoujia.com/apps/7894236'
#url2 ='https://www.wandoujia.com/apps/7596952'
#url2 = 'https://www.wandoujia.com/apps/7970270'
url2 = 'https://www.wandoujia.com/apps/6586136'
url2 = 'https://www.wandoujia.com/apps/280379'

from lxml import html
import requests
from requests.utils import quote
response = requests.get(url2)
tree = html.fromstring(response.content)
#desc = tree.xpath('/html/body/div[2]/div[2]/div[2]/div[1]/div[3]/div/div/text()[1]'
#desc = tree.xpath('/html/body/div[2]/div[2]/div[2]/div[1]/div/div/div[1]/p/text()[1]')
desc = tree.xpath('/html/body/div[2]/div[2]/div[2]/div[1]/div/div/div[1]/*/text()')
desc = tree.xpath('/html/body/div[2]/div[2]/div[2]/div/div/div/*/text()')
desc = tree.xpath('/html/body/div[2]/div[2]/div[2]/div/*/div[@itemprop="description"]//text()')#[0]

dp = tree.xpath('/html/body/div[2]/div[2]/div[2]//div[@class="editorComment"]/div[@class="con"]/text()')  # [0]
dp = dp[0] if len(dp) > 0 else 'No desc'
print('dp', dp)

print('desc', desc)
# print(dir(desc))
# print(desc)
# print('hello')

# desc = ''.join(desc) if len(desc) > 0 else 'No desc'
# desc = desc.replace('\t','')
# desc = desc.replace('\r','')
# desc = desc.replace('\n','')
# print(desc)
# #html = f'http://app.mi.com/details?id={pkg}'

2019-08-30 12:05:28,614 connectionpool.py[815] DEBUG Starting new HTTPS connection (1): www.wandoujia.com:443
2019-08-30 12:05:29,565 connectionpool.py[396] DEBUG https://www.wandoujia.com:443 "GET /apps/280379 HTTP/1.1" 200 None


dp   中国电信推出的移动支付手机客户端，主要功能和同类 app 差不多，转账存款、购物商城、水电支付、预定车票等等，还是可以试试看的。  
desc ['翼支付是一款综合性移动支付业务和理财服务的应用, 致力于打造全国优质的掌上生活消费平台，多重支付保障体系，助您尽享便利生活服务。【微信公众号】甜橙金融【主要功能】1.生活缴费：水电煤缴费、有线电视缴费、交罚代交等；2.通信缴费：手机话费充值、流量充值、固话缴费、宽带缴费等；3.生活支付：免费异地跨行转账（知道手机号码即可转账）、信用卡还款、电影票、火车票、红包等服务；4.本地化生活服务：O2O商超、加油支付服务，周五六日常态95折+更多营销活动，折扣多多，便利十足；5.理财业务：联合金融机构为用户活期、定期、基金、股票、黄金等理财服务，截至目前已为用户赚取收益超30亿元！历史兑付率100%；6.校园业务：学生卡充值服务，支持校园周边商超购物支付服务，为学生炫酷的手机刷卡服务。期待您的反馈：我们每时每刻都在注意聆听您的意见与任何反馈，翼支付客户端并不能在每一款Android设备上完美的运行，如果它出现了任何的问题，请您发送一封电子邮件给我们，我们将会为此问题找到一个好的解决方案！感谢您使用翼支付业务！kefu@bestpay.com.cn', '  ', '翼支付2019更新内容', '感谢您支持翼支付，本期功能优化如下：1、「任务中心」重磅，动动手指完成本月任务，大额红包领不停；2、银行卡二类开户流程优化，开通、换绑更便捷；3、优化了首页，财富页等UI细节，页面视觉更清晰；4、信用卡还款0手续费；5、更多体验细节优化，等你发现。', '  ']


In [277]:
!pwd
tmp = pd.concat([pd.read_hdf(file, 'wdj') for file in glob('./output/spider/*.h5')])
print(tmp.shape)
tmp.loc[tmp.name=='XBrowser']

/Users/lali2/Documents/workspace_py/xf_tag
(1562874, 11)


,cat_list,closed_ids,ct,desc,dp,match_name,name,path,source,tag_list,wd_id
75791,NaN,NaN,2019-08-29 11:02:37.896044,NaN,NaN,NaN,XBrowser,NaN,NaN,NaN,NaN
74851,NaN,NaN,2019-08-30 01:38:23.271466,"XBrowser安卓版是一款简洁大气的手机浏览器,”X”源于单词eXtreme(极端的),极...",NaN,NaN,XBrowser,NaN,bd,NaN,NaN
74974,NaN,NaN,2019-08-30 01:38:23.271466,"XBrowser安卓版是一款简洁大气的手机浏览器,”X”源于单词eXtreme(极端的),极...",NaN,NaN,XBrowser,NaN,bd,NaN,NaN
74974,NaN,NaN,2019-08-30 01:38:23.271466,"XBrowser安卓版是一款简洁大气的手机浏览器,”X”源于单词eXtreme(极端的),极...",NaN,NaN,XBrowser,NaN,bd,NaN,NaN
74974,NaN,NaN,2019-08-30 01:38:23.271466,"XBrowser安卓版是一款简洁大气的手机浏览器,”X”源于单词eXtreme(极端的),极...",NaN,NaN,XBrowser,NaN,bd,NaN,NaN
74974,NaN,NaN,2019-08-30 01:38:23.271466,"XBrowser安卓版是一款简洁大气的手机浏览器,”X”源于单词eXtreme(极端的),极...",NaN,NaN,XBrowser,NaN,bd,NaN,NaN


In [285]:
tmp.ct = tmp.ct.astype(str)
good_df = tmp.sort_values('ct').drop_duplicates('name', keep='last')
#good_df = good_df.loc[~((good_df.desc=='No desc') | (good_df.desc.str.len()<=20) | pd.isna(good_df.desc))]
good_df.head()


,cat_list,closed_ids,ct,desc,dp,match_name,name,path,source,tag_list,wd_id
0,"办公商务,笔记","6753331,6631597,6584885,6566707",1566920072.884119,清除干扰，刚刚好。 (OFFTIME) 统计并个性化 你的通讯，让你能专心于重要的事物。清除...,拔掉，就足够了！,断线时间OFFTIME,( OFFTIME ),NaN,NaN,"战斗 , 恐怖 , 阅读学习 , 常用工具 , 系统工具 , 电子书...",6518095
1,"通讯社交,电话通讯","6516344,6503396,6547028,280787",1566956853.496439,简单的应用程序显示在您的设备中的SIM卡的信息。它也显示保存在SIM卡上的联系人。 按选定的...,对于SIM卡上的联系人进行增删改查等操作,SIM卡,(U)SIM卡应用,NaN,NaN,,2230555
2,"系统工具,安全","7949692,566489,6618765,596157",1566956856.41078,★QQ安全中心，用心守护您的QQ★当您的帐号出现安全风险时；当您忘记了QQ密码时；当您消费Q...,有了它，再也不用担心QQ，微信，游戏账号被黑被盗。,QQ安全中心,(推荐)安全安装,NaN,NaN,"安全工具 , 腾讯 , 系统安全 , 密码",280221
3,"系统工具,优化","6597626,827557,138133,6767113",1566956860.0010839,现在短信多为通知类短信，多而杂，批量删除又担心误删，不删又杂乱无章？有用短信找寻困难，清理垃...,专注有用短信归纳,短信夹,0 类信息,NaN,NaN,,7600827
4,"通讯社交,聊天,电话通讯","6597626,2316991,138133,34555",1566956861.4199538,GO短信加强版，全球最受用户欢迎的实用手机短信应用，加强系统短信锁不具备的众多功能。主题换肤...,安全锁、私密箱、骚扰拦截，短信从此更安全,GO短信,0 类短信,NaN,NaN,"娱乐 , 小清新 , 文字 , 涂鸦 , 表情 , 备份 , 通...",280253


In [286]:
good_df.loc[good_df.name=='WisdomEye']
good_df.loc[good_df.name=='XBrowser']

,cat_list,closed_ids,ct,desc,dp,match_name,name,path,source,tag_list,wd_id
74851,NaN,NaN,2019-08-30 01:38:23.271466,"XBrowser安卓版是一款简洁大气的手机浏览器,”X”源于单词eXtreme(极端的),极...",NaN,NaN,XBrowser,NaN,bd,NaN,NaN


In [272]:
tmp.loc[tmp.name=='WiFiAnalyzer']#.sort_values('ct').drop_duplicates('name', keep='last')

,cat_list,closed_ids,ct,desc,dp,match_name,name,path,source,tag_list,wd_id
75762,NaN,NaN,2019-08-29 11:02:11.620887,NaN,NaN,NaN,WiFiAnalyzer,NaN,NaN,NaN,NaN


In [288]:

good_df['len_'] = good_df.desc.str.len() #.value_counts()
good_df.loc[(good_df.len_>8) &(good_df.len_<20)]


,cat_list,closed_ids,ct,desc,dp,match_name,name,path,source,tag_list,wd_id,len_
75968,"影音播放,视频","6652276,7328316,2267297,258465",2019-08-30 01:57:15.383407,百视通是一款网络视频软件。,No desc,百视通 BesTV,百視通,软件分类,wdj,,94821,15
75968,"跑酷竞速,赛车","7685069,7711012,7911780,7841882",2019-08-30 01:57:15.747490,Super Dario,No desc,Super Dario,Super Token,游戏分类,wdj,,7967051,13
75969,"跑酷竞速,赛车","7665755,7609228,7699165,7663277",2019-08-30 01:57:16.979994,Robot Space,No desc,Robot Space,Supercar Robot,游戏分类,wdj,,7904899,13
75969,"影音播放,视频","6652276,7328316,2267297,258465",2019-08-30 01:57:17.485008,百视通是一款网络视频软件。,No desc,百视通 BesTV,百視通TV,软件分类,wdj,,94821,15
75970,"影音播放,视频","6652276,7328316,2267297,258465",2019-08-30 01:57:19.848138,百视通是一款网络视频软件。,No desc,百视通 BesTV,百视通,软件分类,wdj,,94821,15
75971,"办公商务,笔记","8003643,7619108,255811,7950036",2019-08-30 01:57:20.607456,Light MTR Map,No desc,MTR Map,T map,软件分类,wdj,,6691172,15
75971,"影音播放,视频","6652276,7328316,2267297,258465",2019-08-30 01:57:22.174264,百视通是一款网络视频软件。,No desc,百视通 BesTV,百视通TV,软件分类,wdj,,94821,15
75973,"经营策略,养成","7603678,6809559,7697427,7943720",2019-08-30 01:57:23.346459,save Tom cat,No desc,save Tom cat,Talking Tom Cat - Android S2S,游戏分类,wdj,,7965123,14
75972,"影音播放,视频","6652276,7328316,2267297,258465",2019-08-30 01:57:23.954683,百视通是一款网络视频软件。,No desc,百视通 BesTV,百视通少儿,软件分类,wdj,,94821,15
75974,"动作冒险,街机","609571,6995119,7945463,7953096",2019-08-30 01:57:25.778966,Target13,No desc,Target13,Target,游戏分类,wdj,,7932377,10


In [303]:

def get_data_from_bd(name):
    url2 = f'http://www.baidu.com/s?wd={name}%20安卓'
    from lxml import html
    import requests
    from requests.utils import quote

    response = requests.get(url2)
    # print(response.text)
    tree = html.fromstring(response.content)
    # print(tree)
    # print(url2)
    # print(response.text)
    # desc = tree.xpath('/html/body/div[2]/div[2]/div[2]/div[1]/div[3]/div/div/text()[1]'
    # desc = tree.xpath('/html/body/div[2]/div[2]/div[2]/div[1]/div/div/div[1]/p/text()[1]')
    desc = tree.xpath('//*[@id="1" or @id="2"]//div[@class="c-abstract"]//text()')

    desc = ''.join(desc)
    import re
    desc = re.sub('[0-9]*年[0-9]*月[0-9]*日.-.', '', desc)

    res = {
        'name': name,
        'desc': desc,
        'ct': pd.to_datetime('now'),
        'source': 'bd'

    }
    return desc


def extend_df():
    tqdm.pandas(desc="my bar!")
    good_df = get_final()
    good_df.desc = good_df.desc.str.strip()
    todo = good_df.loc[(
                        (good_df.desc.str.len() <= 20) |
                        pd.isna(good_df.desc))
    ]
    print(len(todo))
    desc_bd = todo.name.progress_apply(lambda name: get_data_from_bd(name))
    good_df['desc_bd'] = desc_bd

    good_df.to_hdf('./input/final.h5', 'desc')

def get_final():
    if os.path.exists('./input/final.h5'):
        return  pd.read_hdf('./input/final.h5','desc')
    else:
        good_df = pd.concat([pd.read_hdf(file, 'wdj') for file in glob('./output/spider/*.h5')])
        good_df.ct = good_df.ct.astype(str)
        good_df = good_df.sort_values('ct').drop_duplicates('name', keep='last')
        good_df = good_df.reset_index(drop=True)
        return good_df

    
extend_df()

my bar!:   0%|          | 0/19090 [00:00<?, ?it/s]

19090


2019-08-30 11:21:00,665 connectionpool.py[206] DEBUG Starting new HTTP connection (1): www.baidu.com:80
2019-08-30 11:21:01,316 connectionpool.py[396] DEBUG http://www.baidu.com:80 "GET /s?wd=008%E7%A5%9E%E5%99%A80113%20%E5%AE%89%E5%8D%93 HTTP/1.1" 200 None
my bar!:   0%|          | 2/19090 [00:02<6:09:02,  1.16s/it]2019-08-30 11:21:01,896 connectionpool.py[206] DEBUG Starting new HTTP connection (1): www.baidu.com:80
2019-08-30 11:21:02,610 connectionpool.py[396] DEBUG http://www.baidu.com:80 "GET /s?wd=008%E7%A5%9E%E5%99%A8A%20%E5%AE%89%E5%8D%93 HTTP/1.1" 200 None
my bar!:   0%|          | 3/19090 [00:03<6:10:58,  1.17s/it]2019-08-30 11:21:03,076 connectionpool.py[206] DEBUG Starting new HTTP connection (1): www.baidu.com:80
2019-08-30 11:21:03,858 connectionpool.py[396] DEBUG http://www.baidu.com:80 "GET /s?wd=2019%E5%AD%A6%E7%94%9F%E7%94%B5%E8%84%91%20%E5%AE%89%E5%8D%93 HTTP/1.1" 200 None
my bar!:   0%|          | 4/19090 [00:04<6:31:17,  1.23s/it]2019-08-30 11:21:04,456 connection

my bar!:   0%|          | 27/19090 [00:34<7:07:19,  1.34s/it]2019-08-30 11:21:34,082 connectionpool.py[206] DEBUG Starting new HTTP connection (1): www.baidu.com:80
2019-08-30 11:21:34,993 connectionpool.py[396] DEBUG http://www.baidu.com:80 "GET /s?wd=4399%E7%8E%8B%E8%80%85%E8%8D%A3%E8%80%80%E5%8A%A9%E6%89%8B%20%E5%AE%89%E5%8D%93 HTTP/1.1" 200 None
my bar!:   0%|          | 28/19090 [00:35<7:08:21,  1.35s/it]2019-08-30 11:21:35,438 connectionpool.py[206] DEBUG Starting new HTTP connection (1): www.baidu.com:80
2019-08-30 11:21:36,348 connectionpool.py[396] DEBUG http://www.baidu.com:80 "GET /s?wd=4399%E7%BE%A4%E7%BB%84%20%E5%AE%89%E5%8D%93 HTTP/1.1" 200 None
my bar!:   0%|          | 29/19090 [00:37<7:13:38,  1.36s/it]2019-08-30 11:21:36,844 connectionpool.py[206] DEBUG Starting new HTTP connection (1): www.baidu.com:80
2019-08-30 11:21:37,591 connectionpool.py[396] DEBUG http://www.baidu.com:80 "GET /s?wd=4399%E8%BF%B7%E4%BD%A0%E4%B8%96%E7%95%8C%E5%8A%A9%E6%89%8B%20%E5%AE%89%E5%8D%93

2019-08-30 11:22:06,114 connectionpool.py[396] DEBUG http://www.baidu.com:80 "GET /s?wd=4G%E7%AE%A1%E5%AE%B6%20%E5%AE%89%E5%8D%93 HTTP/1.1" 200 None
my bar!:   0%|          | 53/19090 [01:06<6:40:34,  1.26s/it]2019-08-30 11:22:06,499 connectionpool.py[206] DEBUG Starting new HTTP connection (1): www.baidu.com:80
2019-08-30 11:22:07,278 connectionpool.py[396] DEBUG http://www.baidu.com:80 "GET /s?wd=4G%E7%BD%91%E7%BB%9C%E7%94%B5%E8%AF%9D%20%E5%AE%89%E5%8D%93 HTTP/1.1" 200 None
my bar!:   0%|          | 54/19090 [01:08<6:30:16,  1.23s/it]2019-08-30 11:22:07,655 connectionpool.py[206] DEBUG Starting new HTTP connection (1): www.baidu.com:80
2019-08-30 11:22:08,520 connectionpool.py[396] DEBUG http://www.baidu.com:80 "GET /s?wd=4G%E9%80%9A%E7%BD%91%E7%BB%9C%E7%94%B5%E8%AF%9D%20%E5%AE%89%E5%8D%93 HTTP/1.1" 200 None
my bar!:   0%|          | 55/19090 [01:09<6:31:52,  1.24s/it]2019-08-30 11:22:08,901 connectionpool.py[206] DEBUG Starting new HTTP connection (1): www.baidu.com:80
2019-08-30 11

KeyboardInterrupt: 

In [295]:
tmp = pd.read_hdf('./input/final.h5','desc')
tmp.loc[pd.notna(tmp.desc_bd)]

,cat_list,closed_ids,ct,desc,dp,match_name,name,path,source,tag_list,wd_id,desc_bd
75968,"影音播放,视频","6652276,7328316,2267297,258465",2019-08-30 01:57:15.383407,百视通是一款网络视频软件。,No desc,百视通 BesTV,百視通,软件分类,wdj,,94821,
75969,NaN,NaN,2019-08-30 01:57:15.721310,,NaN,NaN,#ffababab,NaN,bd,NaN,NaN,
75970,"跑酷竞速,赛车","7685069,7711012,7911780,7841882",2019-08-30 01:57:15.747490,Super Dario,No desc,Super Dario,Super Token,游戏分类,wdj,,7967051,Super Token超级钱包 0.64安卓版软件大小:24.4M 更新时间:2019-01...
75971,"经营策略,养成","6601591,6742464,6596982,7612785",2019-08-30 01:57:16.665540,女友养成计划,No desc,女友养成计划,女神成长计划,游戏分类,wdj,,7637572,"女神养成计划是一款经典的养成类休闲游戏,在游戏中玩家需要进行对女神养成。体验养成的乐趣,感受..."
75972,"跑酷竞速,赛车","7665755,7609228,7699165,7663277",2019-08-30 01:57:16.979994,Robot Space,No desc,Robot Space,Supercar Robot,游戏分类,wdj,,7904899,游戏语言:中文 版本:v1.0 安卓汉化版 更新时间:2017-10-30 11:08 游戏...
75973,NaN,NaN,2019-08-30 01:57:17.046544,,NaN,NaN,13078971364,NaN,bd,NaN,NaN,
75974,"影音播放,视频","6652276,7328316,2267297,258465",2019-08-30 01:57:17.485008,百视通是一款网络视频软件。,No desc,百视通 BesTV,百視通TV,软件分类,wdj,,94821,"百视通TVBesTV_LiteI_DBSC_7.1.1809.5,百视通TV全新改版!智能频..."
75975,"通讯社交,婚恋","38127,7399537,6756079,7684107",2019-08-30 01:57:17.830334,No desc,No desc,好友批量发,好友发收藏,软件分类,wdj,,7539111,安卓微信收藏语音这么转发给苹果好友微信是我们现在经常使用的社交软件之一。今天小编就告诉大家手...
75976,NaN,NaN,2019-08-30 01:57:18.455629,,NaN,NaN,13524529922,NaN,bd,NaN,NaN,
75977,"通讯社交,婚恋","38127,7399537,6756079,7684107",2019-08-30 01:57:18.893217,No desc,No desc,好友批量发,好友批量发,软件分类,wdj,,7539111,"好友批量发最新版截图 网友评论更多 下载豌豆荚,参与网友评论互动 暂无评论 相关信息 大小..."


In [10]:
tmp = pd.read_hdf('./input/final.h5','desc')
# tmp =  tmp.fillna('')
# tmp.desc = tmp.desc.str.strip()
tmp.loc[(tmp.desc.str.len()>10) & (tmp.desc.str.strip().str.len()<10)] 


,cat_list,closed_ids,ct,desc,dp,match_name,name,path,source,tag_list,wd_id,desc_bd
808,"生活休闲,小工具","6764952,7241726,6818380,6618346",2019-08-28 04:16:34.668862,,No desc,008神器工具箱,008神器0113,NaN,NaN,,7559445,NaN
809,"生活休闲,小工具","6764952,7241726,6818380,6618346",2019-08-28 04:16:36.170497,,No desc,008神器工具箱,008神器A,NaN,NaN,,7559445,NaN
813,"办公商务,效率办公",,2019-08-28 04:16:50.622059,,No desc,学生电脑,2019学生电脑,NaN,NaN,,7510675,NaN
817,"金融理财,支付","6571157,28087,66941,596157",2019-08-28 04:17:00.572246,,中国电信推出的移动支付手机客户端，主要功能和同类 app 差不多，转账存款、购物商城、水电支...,翼支付,3K游戏快捷支付,NaN,NaN,,280379,NaN
818,"办公商务,效率办公",,2019-08-28 04:17:02.724077,...,No desc,3Meter,3Meter,NaN,NaN,,6987918,NaN
819,"摄影图像,美化","7683852,6763951,6770523,6503416",2019-08-28 04:17:04.257617,,高端约社交，国内最大街拍图片分享社区！,P1,3Q_P1_Dui,NaN,NaN,,1560045,NaN
821,"网上购物,商城",,2019-08-28 04:17:07.748077,,价格不再千篇一律：每款车都有当地4S店多位销售顾问提供的差异化优惠报价。比价全城，总有你要的低价,买车宝,3Z车宝,NaN,NaN,,321553,NaN
822,"金融理财,支付","6571157,28087,66941,596157",2019-08-28 04:17:09.517098,,中国电信推出的移动支付手机客户端，主要功能和同类 app 差不多，转账存款、购物商城、水电支...,翼支付,3k玩快捷支付,NaN,NaN,,280379,NaN
823,"考试学习,学习",,2019-08-28 04:17:11.341383,,数字组成,4和5的组成,3、4、5的形成,NaN,NaN,,6905664,NaN
824,"健康运动,减肥健身","7744376,7612283,8033407,8022710",2019-08-28 04:17:13.048536,,习惯养成记,3个目标,3个目标,NaN,NaN,,7701826,NaN


In [306]:
tmp.loc[(tmp.desc.str.len()+ tmp.desc_bd.str.len()  )<20 ].shape

(112, 12)

In [316]:
tmp.loc[(tmp.desc.str.len()+ tmp.desc_bd.str.len()  )<20 ]#.shape

,cat_list,closed_ids,ct,desc,dp,match_name,name,path,source,tag_list,wd_id,desc_bd
808,"生活休闲,小工具","6764952,7241726,6818380,6618346",2019-08-28 04:16:34.668862,,No desc,008神器工具箱,008神器0113,,,,7559445,
809,"生活休闲,小工具","6764952,7241726,6818380,6618346",2019-08-28 04:16:36.170497,,No desc,008神器工具箱,008神器A,,,,7559445,
813,"办公商务,效率办公",,2019-08-28 04:16:50.622059,,No desc,学生电脑,2019学生电脑,,,,7510675,
817,"金融理财,支付","6571157,28087,66941,596157",2019-08-28 04:17:00.572246,,中国电信推出的移动支付手机客户端，主要功能和同类 app 差不多，转账存款、购物商城、水电支...,翼支付,3K游戏快捷支付,,,,280379,
818,"办公商务,效率办公",,2019-08-28 04:17:02.724077,,No desc,3Meter,3Meter,,,,6987918,
819,"摄影图像,美化","7683852,6763951,6770523,6503416",2019-08-28 04:17:04.257617,,高端约社交，国内最大街拍图片分享社区！,P1,3Q_P1_Dui,,,,1560045,
821,"网上购物,商城",,2019-08-28 04:17:07.748077,,价格不再千篇一律：每款车都有当地4S店多位销售顾问提供的差异化优惠报价。比价全城，总有你要的低价,买车宝,3Z车宝,,,,321553,
822,"金融理财,支付","6571157,28087,66941,596157",2019-08-28 04:17:09.517098,,中国电信推出的移动支付手机客户端，主要功能和同类 app 差不多，转账存款、购物商城、水电支...,翼支付,3k玩快捷支付,,,,280379,
823,"考试学习,学习",,2019-08-28 04:17:11.341383,,数字组成,4和5的组成,3、4、5的形成,,,,6905664,
824,"健康运动,减肥健身","7744376,7612283,8033407,8022710",2019-08-28 04:17:13.048536,,习惯养成记,3个目标,3个目标,,,,7701826,


In [317]:
tmp.loc[817, 'desc'].strip()

''

In [196]:
url2 = 'http://www.baidu.com/s?wd=阿吉泰%20安卓'
#url2 = 'http://www.baidu.com/s?wd=埃达之光%20安卓'
url2 = 'http://www.baidu.com/s?wd=红灯区%20安卓'
from lxml import html
import requests
from requests.utils import quote

response = requests.get(url2)
#print(response.text)
tree = html.fromstring(response.content)
#print(tree)
#print(url2)
#print(response.text)
#desc = tree.xpath('/html/body/div[2]/div[2]/div[2]/div[1]/div[3]/div/div/text()[1]'
#desc = tree.xpath('/html/body/div[2]/div[2]/div[2]/div[1]/div/div/div[1]/p/text()[1]')
desc = tree.xpath('//*[@id="rso"]/div/div/div[1]/div/div/div[2]/div/span/text()[2]')
desc = tree.xpath('//*[@id="1"]')
desc = tree.xpath('//*[@id="1"]//div[@class="c-abstract"]//text()')
#//*[@id="1"]/div/div[2]/div[1]/text()[2]

desc =  ''.join(desc)
import re
desc = re.sub('[0-9]*年[0-9]*月[0-9]*日.-.', '', desc)

print(desc)

2019-08-29 18:24:53,667 connectionpool.py[206] DEBUG Starting new HTTP connection (1): www.baidu.com:80
2019-08-29 18:24:53,745 connectionpool.py[396] DEBUG http://www.baidu.com:80 "GET /s?wd=%E7%BA%A2%E7%81%AF%E5%8C%BA%20%E5%AE%89%E5%8D%93 HTTP/1.1" 200 None


安卓版|0MB  下载 软件介绍  红灯区appv1.0是小编非常喜欢的一款视频直播软件,本作每天都会为你找寻你心目中的他/她,粉红美女直播app实时交流,在...


In [224]:
tmp.match_name.astype(str).sort_values

<bound method Series.sort_values of 0        断线时间OFFTIME
1               SIM卡
2             QQ安全中心
3                短信夹
4               GO短信
            ...     
76637            风暴岛
76638            食物语
76639            食物语
76640          饥荒合辑版
76641        香信Civet
Name: match_name, Length: 76642, dtype: object>

In [188]:
import re
re.sub?

In [197]:
good_df = pd.concat([pd.read_hdf(file, 'wdj') for file in glob('./output/spider/*.h5')])

if len(good_df)>0:
    good_df = good_df.drop_duplicates('name')

/Users/lali2/dev/python/anaconda3/envs/tf14/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


In [202]:
good_df.loc[((good_df.desc=='No desc') | pd.isna(good_df.desc))]

,cat_list,closed_ids,ct,desc,dp,match_name,name,path,source,tag_list,wd_id
8041,NaN,NaN,2019-08-28 13:22:39.792769,NaN,NaN,NaN,#ffababab,NaN,NaN,NaN,NaN
8042,NaN,NaN,2019-08-28 13:22:40.742529,NaN,NaN,NaN,.,NaN,NaN,NaN,NaN
8043,NaN,NaN,2019-08-28 13:22:41.695893,NaN,NaN,NaN,13078971364,NaN,NaN,NaN,NaN
8044,NaN,NaN,2019-08-28 13:22:42.669973,NaN,NaN,NaN,13524529922,NaN,NaN,NaN,NaN
8045,NaN,NaN,2019-08-28 13:22:43.704446,NaN,NaN,NaN,13764409917,NaN,NaN,NaN,NaN
8046,NaN,NaN,2019-08-28 13:22:44.812161,NaN,NaN,NaN,15021285639,NaN,NaN,NaN,NaN
8047,NaN,NaN,2019-08-28 13:22:45.946711,NaN,NaN,NaN,15614145815,NaN,NaN,NaN,NaN
8048,NaN,NaN,2019-08-28 13:22:46.933385,NaN,NaN,NaN,15801893317,NaN,NaN,NaN,NaN
8049,NaN,NaN,2019-08-28 13:22:47.910946,NaN,NaN,NaN,17511650461,NaN,NaN,NaN,NaN
8050,NaN,NaN,2019-08-28 13:22:49.090198,NaN,NaN,NaN,17751238824,NaN,NaN,NaN,NaN


In [115]:
response.raw.read()

b''

In [36]:
good_df = tmp
good_df.loc[~((good_df.desc=='No desc') | pd.isna(good_df.desc))]

,name,ct,wd_id,match_name,tag_list,cat_list,closed_ids,desc,dp
4,( OFFTIME ),1.56692e+09,6518095,断线时间OFFTIME,"战斗 , 恐怖 , 阅读学习 , 常用工具 , 系统工具 , 电子书...","办公商务,笔记","6753331,6631597,6584885,6566707",清除干扰，刚刚好。 (OFFTIME) 统计并个性化 你的通讯，让你能专心于重要的事物。清除...,拔掉，就足够了！
11,(U)SIM卡应用,1.56696e+09,2230555,SIM卡,,"通讯社交,电话通讯","6516344,6503396,6547028,280787",简单的应用程序显示在您的设备中的SIM卡的信息。它也显示保存在SIM卡上的联系人。 按选定的...,对于SIM卡上的联系人进行增删改查等操作
12,(推荐)安全安装,1.56696e+09,280221,QQ安全中心,"安全工具 , 腾讯 , 系统安全 , 密码","系统工具,安全","7949692,566489,6618765,596157",★QQ安全中心，用心守护您的QQ★当您的帐号出现安全风险时；当您忘记了QQ密码时；当您消费Q...,有了它，再也不用担心QQ，微信，游戏账号被黑被盗。
14,0 类信息,1.56696e+09,7600827,短信夹,,"系统工具,优化","6597626,827557,138133,6767113",现在短信多为通知类短信，多而杂，批量删除又担心误删，不删又杂乱无章？有用短信找寻困难，清理垃...,专注有用短信归纳
15,0 类短信,1.56696e+09,280253,GO短信,"娱乐 , 小清新 , 文字 , 涂鸦 , 表情 , 备份 , 通...","通讯社交,聊天,电话通讯","6597626,2316991,138133,34555",GO短信加强版，全球最受用户欢迎的实用手机短信应用，加强系统短信锁不具备的众多功能。主题换肤...,安全锁、私密箱、骚扰拦截，短信从此更安全
16,0 類訊息,1.56696e+09,8045128,mini讯息,,"经营策略,养成","8015792,7879821,7817390,7932155",“救命！！！”某年某月某日，你又收到了一条求救信息……尬聊王，啊不对，mini讯息是一款流程...,一款模拟短信收发进行的游戏
17,0013网咖,1.56696e+09,6626883,0013网咖,,生活休闲,,主要功能：【网费充值】充值随机立减赢免单，还有签到红包天天领，上网不要钱【网咖订座】实时看到...,影院式选座，一键预订连座、包间
18,007司机,1.56696e+09,50995,快的司机,"驾驶 , 技巧 , 极速 , 出行 , 打车 , 租车","旅游出行,用车租车","7915635,7593502,7601800,6522898","软件描述:快的打车司机版, 全新登场!学会只需 1 分钟, 每月多赚 2000 元!全国覆盖...",智能的打车信息推送方式，让你不错过每一笔生意
19,007应用锁,1.56696e+09,7635396,007应用锁,,"系统工具,安全","6759583,7853582,6525497,6234283",007应用锁是一款专为安卓手机定制的应用加锁工具，采用系统最高级别的加密保护，是装机必备的隐...,前锁未有的保护
20,008彩票,1.56696e+09,40975,500彩票,"生活应用 , 网上购物","金融理财,彩票","7491223,6695847,7840670,8037903",500APP，覆盖全网热门彩种，全方位为您服务。全网公告、走势图表、资讯预测等数据服务；足篮...,体彩玩家主场，大奖诞生之地！


In [33]:
pd.to_datetime('now')

Timestamp('2019-08-28 03:56:04.093927')

In [62]:
app_desc = pd.read_csv(f'{input_dir}/app_desc.dat', delimiter='\t', header=None,  names =['app_id', 'app_des'])
print(app_desc.shape)
app_desc.head()

(94354, 2)


,app_id,app_des
0,BB29DA6F8167CFC99E0853741C4EB17B,注意]游戏需要在设备上自己的歌曲注意]音乐赛车是一个音乐改编的赛车游戏，你用你自己的音乐比赛...
1,BB2A78EA7AD4945EAF6E38997F6139A3,定位试衣到家是一款基于地理位置，提供试衣到家专属购物体验的互联网平台。购物流程客户在试衣到家...
2,BB2B1604CFA079C289FECF927DFBCE89,"想念一个人,就说出来。记得要下载安卓锁屏才可正常显示锁屏效果哦~更新内容更稳定、更优质，邀您..."
3,BB2C7BD0B0623644183DAD08A89E1D90,闽通宝手机客户端是基于移动互联网的，以公众出行服务为基础，贯彻绿色出行，低碳生活的理念，为出...
4,BB2E1A8F56158E483D7461E930E6332F,风靡全球的DIY照片桌面，干净、流畅，启动提速100，瞬间提升手机性能；更是一亿用户的共同选...


In [63]:
app_type = pd.read_csv(f'{input_dir}/apptype_id_name.txt', delimiter='\t', names =['type_id', 'type_name'] )
print(app_type.shape)
app_type.head()

(152, 2)


,type_id,type_name
0,1401,便捷生活
1,1402,游戏
2,1403,通讯社交
3,1404,阅读
4,1405,工作求职


In [65]:
import csv
apptype_train  = pd.read_csv(f'{input_dir}/apptype_train.dat', sep='\t', 
                             names =['app_id', 'type_id', 'app_des'] , 
                             quoting=3
                            )
print(apptype_train.shape)
apptype_train.head()
#apptype_train.iloc[2,2]

(30000, 3)


,app_id,type_id,app_des
0,00000777CE5B5AA5C1AC94DB8EABE0AC,140203,《游戏王座》使用说明书成分由怪兽卡、魔法卡、陷阱卡合计数千张卡牌以及刺激性、耐久性玩法组成。...
1,0000DEC36E15C27DBFC64AB8208C4B37,140206,更稳定、更优质，邀您一起。
2,0001791406307B1D1CE2BC64A830B7C7,142106,《小钱袋》是一款免费网络版记帐软件，适用于个人记帐、家庭记帐、团队记帐，全程帮您安全记录您财...
3,0002F14825B9CA01653325EEFD69D790,142701,领先的周易服务平台高人汇，汇聚算命大师、风水大师、占卜大师、手相大师、起名大师、算命先生、面...
4,000419D79365331F89399E5F38A91B05,140901,平行空间是一款极简、免费的黑科技双开助手；您可以在平行空间双开微信微博、陌陌、映客、yy等应...


In [66]:
apptype_train.iloc[15945:15980]

,app_id,type_id,app_des
15945,63959834D8FB9D68C03A75C9BB0906EA,140206,在全新的地图中，你将与戴夫一起面对驾驶飞行器呼啸而来的僵尸军团，肩负起守卫天空之城的重任，同...
15946,63961F67B88D3D7D877101F80A53E5CD,140901,部分小错误，整体。
15947,6396C70B6383F0BF243EF69927ACF35F,140901,以太大陆EthMin 以太大陆是一个数字生态世界，帮助个体管理在现实世界中所付出的努力与贡献...
15948,6396F4C27E1F1D86762B9283D701DB78,142501,帮助准妈妈在分娩前记录宫缩频率和时长，以判断是否达到就医标准。遇到问题，可1对1在线咨询产科...
15949,63997AB7F3E277BC0CB1D42C3D8360F4,142103,线上线下优势资源整合 不必四处奔波，专属咨询顾问为您服务。 安心快速无抵押 去繁求简，...
15950,639B889103E0AFD7D23E8C593DB6A6D1,140211,更稳定、更优质，邀您一起。
15951,639BC48DB51B5806B726B392224F0CA8,142102,金钱永不眠一个股票账户，一笔钱投资美股/港股/英股/A股；全球资产配置的一站式股票平台享受一...
15952,639C08D6CA2142E0CFD60E64DFB7C326,140901,文字转语音合成免费语音翻译、文本朗读、红包口令、普通话吆喝广告音频合成，一款专业进行文字转语...
15953,639C9663BB3CABFA048B3A54ED9B8CC9,140401,在微博，官方发布新闻，草根爆料八卦；在微博，大V明星发布动态，粉丝狗仔爆料内幕；在微博，海量...
15954,639DBC25084151D681F73C1A331B6CBA,140210,比斗地主麻将更简单、比炸金花牛牛更刺激，全球百姓共同推荐锻炼情商智商，大奖话费信手拈来。客官...
